## Data input for BIDS datasets
`DataGrabber` and `SelectFiles` are great if you are dealing with generic datasets with arbitrary organization. However if you have decided to use Brain Imaging Data Structure (BIDS) to organized your data (or got your hands on a BIDS dataset) you can take advanted of a formal structure BIDS imposes. In this short tutorial you will learn how to do this.

## `pybids` - a Python API for working with BIDS datasets
`pybids` is a lightweight python API for querying BIDS folder structure for specific files and metadata. You can install it from PyPi:
```
pip install pybids
```
Please note it should be already installed in the tutorial Docker image.

## The `layout` object and simple queries
To begin working with pubids we need to initalize a layout object. We will need it to do all of our queries

In [ ]:
from bids.grabbids import BIDSLayout
layout = BIDSLayout("/data/ds000114/")

In [ ]:
!tree -L 4 /data/ds000114/

Let's figure out what are the subject labels in this dataset

In [ ]:
layout.get_subjects()

What modalities are included in this dataset?

In [ ]:
layout.get_modalities()

What different data types are included in this dataset?

In [ ]:
layout.get_types(modality='func')

What are the different tasks included in this dataset?

In [ ]:
layout.get_tasks()

We can also ask for all of the data for a particular subject and one modality.

In [ ]:
layout.get(subject='01', modality="anat", session="test")

We can also ask for a specific subset of data. Note that we are using extension filter to get just the imaging data (BIDS allows both .nii and .nii.gz so we need to include both).

In [ ]:
layout.get(subject='01', type='bold', extensions=['nii', 'nii.gz'])

You probably noticed that this method does not only return the file paths, but objects with relevant query fields. We can easily extract just the file paths.

In [ ]:
[f.filename for f in layout.get(subject='01', type='bold', extensions=['nii', 'nii.gz'])]

### Exercise 1:
List all files for the "linebisection" task for subject 02.

## Including `pybids` in your `nipype` workflow
This is great, but what we really want is to include this into our `nipype` workflows. How to do this? We can create our own custom `BIDSDataGrabber` using a `Function` Interface. First we need a plain Python function that for a given subject label and dataset location will return list of BOLD files.

In [ ]:
def get_niftis(subject_id, data_dir):
    # Remember that all the necesary imports need to be INSIDE the function for the Function Interface to work!
    from bids.grabbids import BIDSLayout
    
    layout = BIDSLayout(data_dir)
    
    bolds = [f.filename for f in layout.get(subject=subject_id, type="bold", extensions=['nii', 'nii.gz'])]
    
    return bolds

In [ ]:
get_niftis('01', '/data/ds000114')

Ok we got our function. Now we need to wrap it inside a Node object.

In [ ]:
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.utility import IdentityInterface, Function

In [ ]:
BIDSDataGrabber = Node(Function(function=get_niftis, input_names=["subject_id",
                                       "data_dir"],
                                   output_names=["bolds"]), name="BIDSDataGrabber")
BIDSDataGrabber.inputs.data_dir = "/data/ds000114"

In [ ]:
BIDSDataGrabber.inputs.subject_id='01'
res = BIDSDataGrabber.run()
res.outputs

Works like a charm! (hopefully :) Lets put it in a workflow. We are not going to analyze any data, but for demostrantion purposes we will add a couple of nodes that pretend to analyze their inputs

In [ ]:
def printMe(paths):
    print("\n\nanalyzing " + str(paths) + "\n\n")
    
analyzeBOLD = Node(Function(function=printMe, input_names=["paths"],
                            output_names=[]), name="analyzeBOLD")

In [ ]:
wf = Workflow(name="bids_demo")
wf.connect(BIDSDataGrabber, "bolds", analyzeBOLD, "paths")
wf.run()

### Exercise 2:
Modify the `BIDSDataGrabber` and the workflow to include T1ws.

## Iterating over subject labels
In the previous example we demostrated how to use `pybids` to "analyze" one subject. How can we scale it for all subjects? Easy - using `iterables` (more in [Iteration/Iterables](basic_iteration.ipynb).

In [ ]:
BIDSDataGrabber.iterables = ('subject_id', layout.get_subjects()[:2])
wf.run()

## Accessing additional metadata
Querying different files is nice, but sometimes you want to access more metadata. For example `RepetitionTime`. `pybids` can help with that as well

In [ ]:
layout.get_metadata('/data/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-fingerfootlips_bold.nii.gz')

Can we incorporate this into our pipeline? Yes we can!
(More about MapNode in [MapNode](basic_mapnodes.ipynb))

In [ ]:
def printMetadata(path, data_dir):
    from bids.grabbids import BIDSLayout
    layout = BIDSLayout(data_dir)
    print("\n\nanalyzing " + path + "\nTR: "+ str(layout.get_metadata(path)["RepetitionTime"]) + "\n\n")
    
analyzeBOLD2 = MapNode(Function(function=printMetadata, input_names=["path", "data_dir"],
                             output_names=[]), name="analyzeBOLD2", iterfield="path")
analyzeBOLD2.inputs.data_dir = "/data/ds000114/"

In [ ]:
wf = Workflow(name="bids_demo")
wf.connect(BIDSDataGrabber, "bolds", analyzeBOLD2, "path")
wf.run()

### Exercise 3:
Modify the `printMetadata` function to also print `EchoTime` 